### libraries
----------------------------------

In [ ]:
import torch, torchvision
import os
from google.colab import files
from torchvision import transforms
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import models
from torch import optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import numpy as np
import shutil
from PIL import Image
from tqdm import tqdm


# mount to drive
--------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


------------------------------------------
#connect to kaggle for dataset
----------------------------------------

In [ ]:
# Upload the Kaggle API token to Colab
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohamedeslammohamed","key":"534826d15fd337218e097a5729814682"}'}

In [ ]:
#This allows Colab to authenticate with Kaggle.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Download the dataset from Kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 96% 2.21G/2.29G [00:29<00:01, 47.3MB/s]
100% 2.29G/2.29G [00:29<00:00, 83.7MB/s]


In [ ]:
# Unzip the dataset
!unzip -q chest-xray-pneumonia.zip -d /content/


In [ ]:
# Confirm dataset structure
base = "/content/chest_xray"

for split in ["train","val","test"]:
    for cls in os.listdir(os.path.join(base, split)):
        path = os.path.join(base, split, cls)
        print(split, cls, len(os.listdir(path)))


train PNEUMONIA 3875
train NORMAL 1341
val PNEUMONIA 8
val NORMAL 8
test PNEUMONIA 390
test NORMAL 234


--------------------------------
# preprocessing for vlidation and test
----------------------------------

In [ ]:
# Preprocessing transforms
# Why: DenseNet121 expects 3-channel images normalized to ImageNet statistics and a fixed size (commonly 224×224).
IMG_SIZE = 224

val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),   # if images are single-channel
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])


In [ ]:
img = Image.open("/content/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg").convert('L')  # grayscale
t = val_transforms(img)
print(t.shape, t.min().item(), t.max().item())


torch.Size([3, 224, 224]) -2.1179039478302 2.465708017349243


----------------------------------------
# preprocessing for train
---------------------------------------

In [ ]:
# Increase robustness and reduce overfitting while preserving medically meaningful features.
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.9, 1.0), ratio=(0.9,1.1)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),  # small rotation
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])


In [ ]:
#  Define datasets
train_ds = ImageFolder(os.path.join(base, "train"), transform=train_transforms)
val_ds   = ImageFolder(os.path.join(base, "val"),   transform=val_transforms)
test_ds  = ImageFolder(os.path.join(base, "test"),  transform=val_transforms)

#  Batch size (adjust if out of memory)
BATCH_SIZE = 16  # try 8 if T4 runs out of RAM

#  Create data loaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

#  Test one batch
imgs, labels = next(iter(train_loader))
print(imgs.shape, labels.shape)


torch.Size([16, 3, 224, 224]) torch.Size([16])


-------------------------------------------------
# upload the data set of the unknown images
-----------------------------------------------

In [ ]:
# --- CONFIGURATION ---
# Source path of your existing X-ray data (Standard Colab/Kaggle path)
SOURCE_XRAY_ROOT = "/content/chest_xray"

# Destination path for the new clean 3-class dataset
NEW_DATASET_ROOT = "/content/three_class_dataset"

# Number of 'Unknown' images to generate for each split
COUNTS = {
    "train": 2000,  # Add 5000 unknown images to training
    "test": 600,    # Add 600 unknown images to testing
    "val": 50       # Add 50 unknown images to validation
}

# ---------------------

def setup_directories():
    """Creates the train/val/test directories for the new dataset."""
    for split in ["train", "test", "val"]:
        for cls in ["NORMAL", "PNEUMONIA", "UNKNOWN"]:
            os.makedirs(os.path.join(NEW_DATASET_ROOT, split, cls), exist_ok=True)

def copy_xrays():
    """Copies existing X-ray images to the new dataset structure."""
    print("\n--- Copying X-Ray Images (Normal & Pneumonia) ---")

    splits = ["train", "test", "val"]
    classes = ["NORMAL", "PNEUMONIA"]

    for split in splits:
        for cls in classes:
            src_dir = os.path.join(SOURCE_XRAY_ROOT, split, cls)
            dst_dir = os.path.join(NEW_DATASET_ROOT, split, cls)

            if not os.path.exists(src_dir):
                print(f"Warning: Source directory not found: {src_dir}")
                continue

            # Copy files
            files = os.listdir(src_dir)
            print(f"Copying {len(files)} images from {split}/{cls}...")
            for f in files:
                src_file = os.path.join(src_dir, f)
                dst_file = os.path.join(dst_dir, f)
                if os.path.isfile(src_file):
                    shutil.copy(src_file, dst_file)

def create_unknown_data():
    """Downloads CIFAR-100 and distributes it as 'UNKNOWN' class."""
    print("\n--- Generating 'UNKNOWN' Class from CIFAR-100 ---")

    # Load CIFAR-100 (We use train set for everything to get enough quantity)
    cifar_ds = datasets.CIFAR100(root='./data_cifar', train=True, download=True)

    total_idx = 0

    for split, count in COUNTS.items():
        print(f"Creating {count} unknown images for {split} set...")
        save_dir = os.path.join(NEW_DATASET_ROOT, split, "UNKNOWN")

        for i in range(count):
            # Get image from CIFAR
            img, _ = cifar_ds[total_idx]

            # Save as JPEG
            save_path = os.path.join(save_dir, f"unknown_{total_idx:05d}.jpg")
            img.save(save_path)

            total_idx += 1

            # Safety break if we run out of CIFAR images (CIFAR train has 50k)
            if total_idx >= len(cifar_ds):
                break

# --- EXECUTION ---
if __name__ == "__main__":
    setup_directories()
    copy_xrays()
    create_unknown_data()

    # Clean up CIFAR download cache
    if os.path.exists('./data_cifar'):
        shutil.rmtree('./data_cifar')

    print(f"\nSUCCESS: Dataset ready at {NEW_DATASET_ROOT}")
    print("Classes present: NORMAL, PNEUMONIA, UNKNOWN")


--- Copying X-Ray Images (Normal & Pneumonia) ---
Copying 1341 images from train/NORMAL...
Copying 3875 images from train/PNEUMONIA...
Copying 234 images from test/NORMAL...
Copying 390 images from test/PNEUMONIA...
Copying 8 images from val/NORMAL...
Copying 8 images from val/PNEUMONIA...

--- Generating 'UNKNOWN' Class from CIFAR-100 ---


100%|██████████| 169M/169M [00:04<00:00, 34.5MB/s]


Creating 2000 unknown images for train set...
Creating 600 unknown images for test set...
Creating 50 unknown images for val set...

SUCCESS: Dataset ready at /content/three_class_dataset
Classes present: NORMAL, PNEUMONIA, UNKNOWN


In [ ]:
# --- CONFIGURATION ---
DATASET_ROOT = "/content/three_class_dataset"
BATCH_SIZE = 16

# --- DATASETS & LOADERS ---

def get_data_loaders():
    print("Initializing Data Loaders...")

    # 1. Define Datasets (ImageFolder automatically finds classes by folder name)
    train_ds = datasets.ImageFolder(os.path.join(DATASET_ROOT, "train"), transform=train_transforms)
    val_ds   = datasets.ImageFolder(os.path.join(DATASET_ROOT, "val"), transform=val_transforms)
    test_ds  = datasets.ImageFolder(os.path.join(DATASET_ROOT, "test"), transform=val_transforms)

    # 2. Define Loaders
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # 3. Print Summary
    class_names = train_ds.classes
    print(f"Classes found: {class_names}")
    print(f"Training samples: {len(train_ds)}")
    print(f"Validation samples: {len(val_ds)}")

    return train_loader, val_loader, test_loader, class_names

# --- USAGE ---
train_loader, val_loader, test_loader, class_names = get_data_loaders()

# Verify a single batch
images, labels = next(iter(train_loader))
print(f"\nBatch shape: {images.shape}") # Should be [32, 3, 224, 224]
print(f"Labels shape: {labels.shape}")

Initializing Data Loaders...
Classes found: ['NORMAL', 'PNEUMONIA', 'UNKNOWN']
Training samples: 7216
Validation samples: 66

Batch shape: torch.Size([16, 3, 224, 224])
Labels shape: torch.Size([16])


#TRAIN CLASSIFIER ONLY

In [ ]:

# 1. Load pretrained DenseNet121 (ImageNet weights)
# This downloads the weights if not already cached
model = models.densenet121(pretrained=True)

# 2. Freeze the backbone (Feature Extractor)
# We iterate through the 'features' part of DenseNet and turn off gradient calculation.
# This ensures we don't destroy the pre-trained patterns during the initial training.
for param in model.features.parameters():
    param.requires_grad = False

# 3. Replace final classifier layer for 3 classes
# Classes: 0: Normal, 1: Pneumonia, 2: Unknown (Non-X-ray)
n_features = model.classifier.in_features

# We create a new Linear layer. By default, requires_grad is True for new layers.
model.classifier = nn.Linear(n_features, 3)

# 4. Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 5. Verification: Print total and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Device used: {device}")
print("Model structure modified for 3 classes: [Normal, Pneumonia, Unknown]")
print("-" * 30)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("-" * 30)

if trainable_params < total_params:
    print("SUCCESS: Backbone is frozen. Only the classifier will train.")
else:
    print("WARNING: Backbone appears to be unfrozen.")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 173MB/s]


Device used: cuda
Model structure modified for 3 classes: [Normal, Pneumonia, Unknown]
------------------------------
Total parameters: 6,956,931
Trainable parameters: 3,075
------------------------------
SUCCESS: Backbone is frozen. Only the classifier will train.


In [ ]:
# --- RECALCULATING WEIGHTS FOR 3 CLASSES ---

# Order matches [Normal, Pneumonia, Unknown] (0, 1, 2)
class_weights = torch.tensor([1.78, 0.62, 1.20], dtype=torch.float32).to("cuda")

print(f"Using Class Weights: {class_weights}")

#  Loss function (Weighted for 3 classes)
criterion = nn.CrossEntropyLoss(weight=class_weights)

#  Optimizer — trains only the unfrozen parameters (the classifier)
# Ensure 'model' is the one from phase 1 (where backbone is frozen)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

#  Mixed-precision scaler
scaler = torch.amp.GradScaler('cuda')

NUM_EPOCHS = 4

print(f"Starting training for {NUM_EPOCHS} epochs...")

for epoch in range(NUM_EPOCHS):
    # ---- TRAINING ----
    model.train()
    train_loss = 0.0
    all_preds, all_labels = [], []

    # tqdm gives us a nice progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")

    for imgs, labels in loop:
        imgs, labels = imgs.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()

        # Forward pass (with automatic mixed precision)
        with torch.amp.autocast('cuda'):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Metrics
        train_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1).detach().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

        # Update progress bar
        loop.set_postfix(loss=loss.item())

    # Calculate epoch metrics
    train_loss /= len(train_loader.dataset)
    train_acc = accuracy_score(all_labels, all_preds)

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    all_vpreds, all_vlabels = [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to("cuda"), labels.to("cuda")

            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            vpreds = outputs.argmax(dim=1).cpu().numpy()
            all_vpreds.extend(vpreds)
            all_vlabels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_acc = accuracy_score(all_vlabels, all_vpreds)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}: "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

Using Class Weights: tensor([1.7800, 0.6200, 1.2000], device='cuda:0')
Starting training for 4 epochs...


Epoch 1/4 [Train]: 100%|██████████| 451/451 [02:08<00:00,  3.51it/s, loss=0.382]


Epoch 1/4: Train Loss: 0.6074 | Train Acc: 0.8251 | Val Loss: 0.2409 | Val Acc: 0.9545


Epoch 2/4 [Train]: 100%|██████████| 451/451 [02:07<00:00,  3.53it/s, loss=0.313]


Epoch 2/4: Train Loss: 0.3156 | Train Acc: 0.9254 | Val Loss: 0.1408 | Val Acc: 0.9545


Epoch 3/4 [Train]: 100%|██████████| 451/451 [02:08<00:00,  3.50it/s, loss=0.283]


Epoch 3/4: Train Loss: 0.2436 | Train Acc: 0.9268 | Val Loss: 0.1069 | Val Acc: 0.9545


Epoch 4/4 [Train]: 100%|██████████| 451/451 [02:06<00:00,  3.56it/s, loss=0.116]


Epoch 4/4: Train Loss: 0.2155 | Train Acc: 0.9285 | Val Loss: 0.0945 | Val Acc: 0.9545


In [ ]:
# Collect true labels, hard predictions, AND probabilities for AUC
y_true, y_pred, y_probs = [], [], []
model.eval()

print("Starting evaluation...")

with torch.no_grad():
    for imgs, labels in tqdm(test_loader):
        imgs, labels = imgs.to("cuda"), labels.to("cuda")

        # Use autocast for consistency during inference
        with torch.amp.autocast('cuda'):
            outputs = model(imgs)

        outputs=outputs.float()
        preds = outputs.argmax(dim=1)

        # CHANGED: Calculate Softmax probabilities for ALL 3 classes
        # We need the full shape [Batch_Size, 3] for multi-class evaluation
        probs = torch.softmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_probs.extend(probs.cpu().numpy()) # Store probabilities for AUC

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# --- REPORTING ---
print("\nClassification Report:")
# class_names should be ['NORMAL', 'PNEUMONIA', 'UNKNOWN']
print(classification_report(y_true, y_pred, target_names=class_names))

# --- MULTI-CLASS AUC ---
try:
    # We use 'ovr' (One-vs-Rest) to calculate AUC for 3 classes
    auc_score = roc_auc_score(y_true, y_probs, multi_class='ovr', average='macro')
    print(f"Multi-class AUC (Macro OvR): {auc_score:.4f}")
except Exception as e:
    print(f"Could not calculate AUC: {e}")

Starting evaluation...


100%|██████████| 77/77 [00:15<00:00,  4.92it/s]


Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.64      0.93      0.76       234
   PNEUMONIA       0.94      0.68      0.79       390
     UNKNOWN       1.00      1.00      1.00       600

    accuracy                           0.89      1224
   macro avg       0.86      0.87      0.85      1224
weighted avg       0.91      0.89      0.89      1224

Multi-class AUC (Macro OvR): 0.9826


---------------------------------
# PHASE 2 SETUP (Transition3 + DenseBlock4)
---------------------------------

In [ ]:
import torch
import torch.optim as optim

print("Starting Targeted Fine-Tuning Phase...")

# 1. RE-FREEZE THE ENTIRE MODEL (Safety step)
for param in model.parameters():
    param.requires_grad = False

# 2. UNFREEZE THE CLASSIFIER (Must be trainable)
for param in model.classifier.parameters():
    param.requires_grad = True

# 3. UNFREEZE TARGETED LAYERS (Transition3 + DenseBlock4)
print("Unfreezing transition3, denseblock4, and classifier.")

for param in model.features.transition3.parameters():
    param.requires_grad = True

for param in model.features.denseblock4.parameters():
    param.requires_grad = True

# 4. DEFINE OPTIMIZER
# Uses a smaller learning rate (1e-5) to preserve pre-trained features
FINE_TUNE_LR = 1e-5
fine_tune_optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=FINE_TUNE_LR
)

# 5. CONFIRMATION
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Phase 2 Configured. Total trainable parameters: {trainable_params:,}")

Starting Targeted Fine-Tuning Phase...
Unfreezing transition3, denseblock4, and classifier.
✅ Phase 2 Configured. Total trainable parameters: 2,687,491


In [ ]:
# --- CELL 2: PHASE 2 TRAINING ---
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Configuration
PNEUMONIA_CLASSIFIER_WEIGHTS_PATH = 'best_pneumonia_classifier.pth'
best_val_loss = float('inf')
FINE_TUNE_EPOCHS = 20

# 1. WEIGHTS (Recalculated for Unknown = 2000 samples)
# Normal: 1.78, Pneumonia: 0.62, Unknown: 1.20
class_weights = torch.tensor([1.78, 0.62, 1.20], dtype=torch.float32).to("cuda")

# 2. LOSS & SCALER
criterion = nn.CrossEntropyLoss(weight=class_weights)
scaler = torch.amp.GradScaler('cuda')

print(f"Starting Fine-Tuning for {FINE_TUNE_EPOCHS} epochs...")

for epoch in range(FINE_TUNE_EPOCHS):
    # ---- TRAINING ----
    model.train()
    train_loss = 0.0
    all_preds, all_labels = [], []

    # Fixed: Loop over 'train_loader', not 'train_loader.dataset'
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{FINE_TUNE_EPOCHS}")

    for imgs, labels in loop:
        imgs, labels = imgs.to("cuda"), labels.to("cuda")

        fine_tune_optimizer.zero_grad()

        # Mixed Precision Forward Pass
        with torch.amp.autocast('cuda'):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        # Backward Pass
        scaler.scale(loss).backward()
        scaler.step(fine_tune_optimizer)
        scaler.update()

        # Metrics accumulation
        train_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1).detach().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

        loop.set_postfix(loss=loss.item())

    # Normalize loss by total images
    train_loss /= len(train_loader.dataset)
    train_acc = accuracy_score(all_labels, all_preds)

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    all_vpreds, all_vlabels = [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to("cuda"), labels.to("cuda")
            with torch.amp.autocast('cuda'):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            vpreds = outputs.argmax(dim=1).cpu().numpy()
            all_vpreds.extend(vpreds)
            all_vlabels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_acc = accuracy_score(all_vlabels, all_vpreds)

    # ---- CHECKPOINTING ----
    if val_loss < best_val_loss:
        print(f"\n--> Val Loss improved ({best_val_loss:.4f} -> {val_loss:.4f}). Saving model.")
        best_val_loss = val_loss
        torch.save(model.state_dict(), PNEUMONIA_CLASSIFIER_WEIGHTS_PATH)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

Starting Fine-Tuning for 20 epochs...


Epoch 1/20: 100%|██████████| 451/451 [02:09<00:00,  3.47it/s, loss=0.0446]



--> Val Loss improved (inf -> 0.0585). Saving model.
Epoch 1: Train Loss: 0.0709 Acc: 0.9719 | Val Loss: 0.0585 Acc: 1.0000


Epoch 2/20: 100%|██████████| 451/451 [02:13<00:00,  3.39it/s, loss=0.0548]


Epoch 2: Train Loss: 0.0657 Acc: 0.9739 | Val Loss: 0.0740 Acc: 1.0000


...


--> Val Loss improved (0.0280 -> 0.0252). Saving model.
Epoch 18: Train Loss: 0.0294 Acc: 0.9885 | Val Loss: 0.0252 Acc: 1.0000


Epoch 19/20: 100%|██████████| 451/451 [02:10<00:00,  3.45it/s, loss=0.00105]



--> Val Loss improved (0.0252 -> 0.0216). Saving model.
Epoch 19: Train Loss: 0.0242 Acc: 0.9922 | Val Loss: 0.0216 Acc: 1.0000


Epoch 20/20: 100%|██████████| 451/451 [02:10<00:00,  3.46it/s, loss=0.343]


Epoch 20: Train Loss: 0.0240 Acc: 0.9917 | Val Loss: 0.0445 Acc: 0.9848


In [ ]:
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, roc_auc_score

# --- CONFIGURATION ---
# Path where you saved the best model after Phase 2
PNEUMONIA_CLASSIFIER_WEIGHTS_PATH = 'best_pneumonia_classifier.pth'
# 1. Load the best saved weights into the current model instance
print(f"Loading weights from {PNEUMONIA_CLASSIFIER_WEIGHTS_PATH}...")
try:
    # Ensure the model is ready to receive weights
    # Note: The model structure (3 classes) must match the saved weights
    model.load_state_dict(torch.load(PNEUMONIA_CLASSIFIER_WEIGHTS_PATH))
    print(" Successfully loaded best weights for final testing.")
except FileNotFoundError:
    print(f" Error: Weights file not found at {PNEUMONIA_CLASSIFIER_WEIGHTS_PATH}. Check Phase 2 execution.")
except Exception as e:
    print(f" An error occurred while loading weights: {e}")

# -----------------------

# Cell 14: Testing Loop
print("Starting Final Evaluation on Test Set...")

y_true, y_pred, y_probs = [], [], []
model.eval()

with torch.no_grad():
    for imgs, labels in tqdm(test_loader):
        imgs, labels = imgs.to("cuda"), labels.to("cuda")

        # Use autocast for consistency during inference
        with torch.amp.autocast('cuda'):
            outputs = model(imgs)

        # --- CRITICAL FIX FOR MIXED PRECISION ---
        # Convert to float32 before Softmax to ensure sums = 1.0
        outputs = outputs.float()

        preds = outputs.argmax(dim=1)

        # --- CRITICAL CHANGE FOR 3 CLASSES ---
        # OLD: probs = torch.softmax(outputs, dim=1)[:, 1]  <-- Binary only
        # NEW: Capture probabilities for ALL 3 classes for Multi-class AUC
        probs = torch.softmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_probs.extend(probs.cpu().numpy())

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# --- SAFETY NORMALIZATION ---
# Ensure probabilities sum to exactly 1.0 to satisfy sklearn strict checks
y_probs = y_probs / y_probs.sum(axis=1, keepdims=True)

# --- REPORTING ---
print("\n" + "="*30)
print("FINAL TEST RESULTS")
print("="*30)

# Classification Report
# class_names should be ['NORMAL', 'PNEUMONIA', 'UNKNOWN']
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Multi-Class AUC Calculation
try:
    # Calculate Macro Average AUC (One-vs-Rest)
    auc_score = roc_auc_score(y_true, y_probs, multi_class='ovr', average='macro')
    print(f"Multi-class AUC (Macro OvR): {auc_score:.4f}")
except Exception as e:
    print(f"Could not calculate AUC: {e}")

Loading weights from best_pneumonia_classifier.pth...
✅ Successfully loaded best weights for final testing.
Starting Final Evaluation on Test Set...


100%|██████████| 77/77 [00:15<00:00,  5.13it/s]


FINAL TEST RESULTS

Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.92      0.93      0.92       234
   PNEUMONIA       0.96      0.95      0.95       390
     UNKNOWN       1.00      1.00      1.00       600

    accuracy                           0.97      1224
   macro avg       0.96      0.96      0.96      1224
weighted avg       0.97      0.97      0.97      1224

Multi-class AUC (Macro OvR): 0.9953


-------------------------------
#test the model
----------------------------------

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

# --- CONFIGURATION ---
IMG_SIZE = 224
class_names = ["NORMAL", "PNEUMONIA", "UNKNOWN"]

# # Define the exact preprocessing used during training
# # We use 'val_transforms' logic (No random augmentations)
# val_transforms = transforms.Compose([
#     transforms.Resize((IMG_SIZE, IMG_SIZE)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

def predict_single_image(image_path, model, transform):
    """Loads an image, preprocesses it, and gets the model's prediction."""

    # 1. Load the image
    # IMPORTANT: Convert to RGB.
    # - X-rays will become 3-channel Grayscale (r=g=b).
    # - Unknown images (CIFAR) will keep their color.
    # This difference (Color vs No Color) helps the model detect 'Unknown' images.
    try:
        img = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, 0.0, None

    # 2. Preprocess the image
    input_tensor = transform(img)

    # 3. Add a batch dimension and move to GPU
    # The model expects a batch of images (B, C, H, W), so we add B=1
    input_batch = input_tensor.unsqueeze(0).to("cuda")

    # 4. Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        # Use autocast if the model was trained with mixed precision
        with torch.amp.autocast('cuda'):
            output = model(input_batch)

        # Get probabilities using Softmax
        probabilities = F.softmax(output, dim=1)

    # Get the predicted class index
    predicted_index = torch.argmax(probabilities, dim=1).item()

    # Get the confidence for the predicted class
    confidence = probabilities[0, predicted_index].item()

    return predicted_index, confidence, probabilities.cpu().squeeze()

# --- USAGE ---
# IMPORTANT: Replace this path with the image you want to test
NEW_IMAGE_PATH = "/content/pppennnn.jpg"

print(f"--- Analysis for: {NEW_IMAGE_PATH.split('/')[-1]} ---")

# Run prediction
idx, conf, probs = predict_single_image(NEW_IMAGE_PATH, model, val_transforms)

if idx is not None:
    predicted_class = class_names[idx]

    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence:      {conf:.4f}")
    print("-" * 30)
    print("Detailed Probabilities:")
    print(f"  Normal:    {probs[0]:.4f}")
    print(f"  Pneumonia: {probs[1]:.4f}")
    print(f"  Unknown:   {probs[2]:.4f}")

--- Analysis for: pppennnn.jpg ---
Predicted Class: PNEUMONIA
Confidence:      0.9751
------------------------------
Detailed Probabilities:
  Normal:    0.0032
  Pneumonia: 0.9751
  Unknown:   0.0216


----------------------------------------
#saving the weight of the model to drive
-------------------------------------

In [ ]:
import shutil
import os
from google.colab import drive


# 2. Define paths
# The file currently saved in your Colab temporary session
LOCAL_MODEL_PATH = 'best_pneumonia_classifier.pth'

# The folder in your Google Drive where you want to keep it
# It will create a folder named 'My_Medical_Project_Models' in your Drive
DRIVE_FOLDER = '/content/drive/MyDrive/My_Medical_Project_Models'
DRIVE_MODEL_PATH = os.path.join(DRIVE_FOLDER, 'densenet_pneumonia_final.pth')

# 3. Create the folder in Drive if it doesn't exist
if not os.path.exists(DRIVE_FOLDER):
    os.makedirs(DRIVE_FOLDER)
    print(f"Created folder: {DRIVE_FOLDER}")

# 4. Copy the file
if os.path.exists(LOCAL_MODEL_PATH):
    print(f"Copying {LOCAL_MODEL_PATH} to Google Drive...")
    shutil.copy(LOCAL_MODEL_PATH, DRIVE_MODEL_PATH)

    if os.path.exists(DRIVE_MODEL_PATH):
        print("-" * 40)
        print(f" SUCCESS! Model saved permanently at:")
        print(f"{DRIVE_MODEL_PATH}")
        print("-" * 40)
        print("You can now close Colab safely. The model is in your Drive.")
    else:
        print(" Error: File copy failed.")
else:
    print(f" Error: Could not find {LOCAL_MODEL_PATH}. Did you run the training cell?")

Created folder: /content/drive/MyDrive/My_Medical_Project_Models
Copying best_pneumonia_classifier.pth to Google Drive...
----------------------------------------
✅ SUCCESS! Model saved permanently at:
/content/drive/MyDrive/My_Medical_Project_Models/densenet_pneumonia_final.pth
----------------------------------------
You can now close Colab safely. The model is in your Drive.
